#  Regridding MAIAC data for GEOS-Chem

In this Jupyter Notebook we provide the process for generating a regridded version of processed MAIAC data to fit our needs for GEOS-Chem. The original data in question is fitted for a 0.05° granularity (already transformed from a sinusoidal grid, individual band data handling and quality check processed), while the GEOS-Chem grid is using 0.25° (lat) by 0.3125° (lon) cells. For our use case, we are simply concerned with a rectangular region centered over India (from 0° to 40° N and 55.9375° E to 106.5° E), for the year 2015.  Thus, we need to consolidate the contents of the original grid using a regridding process. In addition, we provide necessary code to visualize the original data and regridded data.

This cell simply makes all the required imports to have the necessary libraries. Beyond default Python libraries,
we will be using PyHDF and NetCDF4 for processing the input HDF4 data and outputting to NetCDF, as well as NumPy
for general data manipulation. Matplotlib and Cartopy are for the visualization process. 

In [ ]:
import sys
import os
from pyhdf.SD import SD, SDC
import numpy as np
import math
from netCDF4 import Dataset 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy

Here, we download the data from the site to our local storage using the CURL utility. Each file should be 311.045622 MB, so you may need to re-run this for select files if the file size does not appear as such - or if the file has errors when trying to open (uncomment the code in the middle to deal with this). Change the variable LOCAL to be the path to where you wish to store the data, and the day_start, day_end and year as needed.

In [ ]:
LOCAL = "/data0/rm3873/regrid_data_005/"
day_start = 1
day_end = 366
year = 2015
expected_size_mb = 311.045622
redownload_mode = False

s1 = "https://portal.nccs.nasa.gov/datashare/maiac/DataRelease/CMG_0.05degree/AOT5km/2015/MAIACCMG.2015"
s3 = ".hdf"

for i in range(day_start,day_end):
    outname = LOCAL + str(year) + "_" + str(i) + ".hdf"
    
    #Uncomment this section if you are running through your downloaded files and checking they are the right size.
    '''
    redownload_mode = True
    size = os.path.getsize(outname)/1000000.0
    print(str(size) + " MB")
    if(size != expected_size_mb):
        print("Re-downloading needed")
    else:
        continue
    '''
    progress_str = "Downloading Day " + str(i) " of " + str(day_end - day_start)
    if(redownload_mode):
        progress_str = "Re-downloading Day " + str(i)
        
    print(progress_str)      
    s2 = str(i)
    if(i < 10):
        s2 = "00" + str(i)
    if(i >= 10 and i < 100):
        s2 = "0" + str(i)
    link = s1+s2+s3
    
    cmd = "curl -o " + outname + " " + link
    os.system(cmd)

Adjust the desired path for the output of the regridding process, both regular and binary 
(0 for missing values, 1 for present values), and sent regular_mode to False if you are doing the
binary process. You can adjust the lat/lon values and grid size as well, for the region and target grid size 
you want (assuming a lat-long grid).

In [ ]:
binary_path = '/data0/rm3873/binary_regridded.nc'
regular_path = '/data0/rm3873/regridded.nc'
regular_mode = True
lat_st = 0
lat_end = 40.25
lat_siz = 0.25
lon_st = 55.9375
lon_end = 106.5
lon_siz = 0.3125

This cell sets up the output NetCDF file. We create variables for the latitude, longitude
to fit our target region, as well as the time, simply representing each day of the year. We
then have our 3D array to represent the regridded data, 1 lat-long grid for each day of the year.

In [ ]:
ncfile = Dataset(regular_path,mode='w',format='NETCDF4_CLASSIC') 
if(not regular_mode):
    ncfile = Dataset(binary_path,mode='w',format='NETCDF4_CLASSIC') 
lat_dim = ncfile.createDimension('lat', 161)     
lon_dim = ncfile.createDimension('lon', 162)
time = ncfile.createDimension('time',day_end-day_start)

lat = ncfile.createVariable('lat', np.float32, ('lat',))
lat.units = 'degrees_north'
lat.long_name = 'latitude'
lon = ncfile.createVariable('lon', np.float32, ('lon',))
lon.units = 'degrees_east'
lon.long_name = 'longitude'
time = ncfile.createVariable('time', np.float64, ('time',))
time.units = 'days of 2015'
time.long_name = 'days_of_the_year'
# Define a 3D variable to hold the data
aot = ncfile.createVariable('aot',np.float64,('time','lat','lon')) # note: unlimited dimension is leftmost

lat[:] = np.arange(lat_st,lat_end,lat_siz)
lon[:] = np.arange(lon_st,lon_end,lon_siz)
time[:] = np.arange(day_start,day_end)

This cell contains the meat of the actual regridding process.

In [ ]:
regridded = np.full((day_end-day_start,161,162), -0.1)
for day in range(day_start,day_end):
    print('Day: ' + str(day))
    fname = LOCAL + str(year) + "_" + str(day) + ".hdf"
    f = SD(fname, SDC.READ)
    sel = f.select('AOT').get()
    MISSING = -28672
    data = np.full((3600,7200), MISSING)
    for m in range(3600):
        for n in range(7200):
            data[m][n] = sel[m][n]
    i=1000
    ct_lat = 0
    lat_str = 5
    threshold = 0.5


    while(i < 1800):
        #print(str((i-1000)/800*100) + '% complete!')
        if(ct_lat == 159):
            lat_str = 3
        if(ct_lat == 160):
            lat_str = 2
        j = 4718
        ct_lon = 0
        lon_str = 6
        while(j < 5725):
            if(ct_lon == 155):
                lon_str = 11

            #print('Bounding box: [' + str(i) + ',' + str(i+lat_str) + ' by ' + str(j) + ',' + str(j+lon_str) + '] for idx (' + str(ct_lat) + ',' + str(ct_lon) + ')')
            cur_row = [m for m in range(i,i+lat_str)] 
            cur_col = [n for n in range(j,j+lon_str)]
            cur_box_idx = np.ix_(cur_row,cur_col)
            cur_box = data[cur_box_idx]

            total_ct = len(cur_box) * len(cur_box[0])
            non_miss = []
            for a in range(len(cur_box)):
                for b in range(len(cur_box[a])):
                    if(cur_box[a][b] != MISSING):
                        non_miss.append(cur_box[a][b]* 0.001)


            cur_val =  -0.1
            if(not regular_mode):
                cur_val = 0
            if(len(non_miss) >= threshold * total_ct):
                non_miss = np.array(non_miss)
                cur_val = np.average(non_miss)
                if(not regular_mode):
                    cur_val = 1

            regridded[day - 1][ct_lat][ct_lon] = cur_val
            ct_lon = ct_lon + 1
            j = j + lon_str


        ct_lat = ct_lat + 1
        i = i + lat_str
aot[::] = regridded
ncfile.close()

In [ ]:
day = 28
fname = "/data0/rm3873/regrid_data_005/2015_" + str(day) + ".hdf"
f = SD(fname, SDC.READ)
sel = f.select('AOT').get()
MISSING = -28672

#lons = np.full((1681-1080,5586-4960),-0.1)
#lats = np.full((1681-1080,5586-4960),-0.1)
#vals = np.full((1681-1080,5586-4960),-0.1)
lons = []
lats = []
vals = []
lats_miss = []
lons_miss = []

for i in range(1080,1681):
    if(i % 50 == 0):
        print(i)
    for j in range(4960,5586):
        lat = 90 - i*0.05
        lon = j*0.05 - 180
        if(sel[i][j] != MISSING):
            sel[i][j] = sel[i][j] * 0.001
            #vals[i-1080][j-4960] = sel[i][j]
            vals.append(sel[i][j])
            lats.append(lat)
            lons.append(lon)
        else:
            lats_miss.append(lat)
            lons_miss.append(lon)
        #lats[i-1080][j-4960] = lat
        #lons[i-1080][j-4960] = lon
        
plt.figure(figsize=(16,16),facecolor='white', dpi=80)
ax = plt.axes(projection=ccrs.Mercator(min_latitude=5,max_latitude=40,central_longitude=84))
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='gray', alpha=0.5, linestyle='--')
ax.set_extent([67,100,5,37])
ax.add_feature(cartopy.feature.LAND)
ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.COASTLINE,linewidth=0.4)
ax.add_feature(cartopy.feature.BORDERS, linestyle=':',linewidth=0.4)
plt.scatter(x=lons,y=lats,c=vals, cmap='viridis',s=4,alpha=1,transform=ccrs.PlateCarree())
cb = plt.colorbar(label="AOT")
plt.scatter(x=lons_miss,y=lats_miss,c='white',s=4,alpha=1,transform=ccrs.PlateCarree())
plt.text(0,0,"White points are missing values", fontsize=20,bbox=dict(facecolor='red', alpha=1))
plt.savefig("raw_data_viz.png")
plt.show()

In [ ]:


MISSING = -0.1

lons = []
lats = []
vals = []
lats_miss = []
lons_miss = []

for i in range(161):
    if(i % 10 == 0):
        print(i)
    for j in range(162):
        lat = i*0.25
        lon = j*0.3125 + 56.25
        if(regridded[i][j] != MISSING):
            vals.append(regridded[i][j])
            lats.append(lat)
            lons.append(lon)
        else:
            lats_miss.append(lat)
            lons_miss.append(lon)
            
plt.figure(figsize=(16,16),facecolor='white', dpi=80)
ax = plt.axes(projection=ccrs.Mercator(min_latitude=5,max_latitude=40,central_longitude=84))
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='gray', alpha=0.5, linestyle='--')
ax.set_extent([67,100,5,37])
ax.add_feature(cartopy.feature.LAND)
ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.COASTLINE,linewidth=0.4)
ax.add_feature(cartopy.feature.BORDERS, linestyle=':',linewidth=0.4)
plt.scatter(x=lons,y=lats,c=vals, cmap='cool',s=4,alpha=1,transform=ccrs.PlateCarree())
cb = plt.colorbar(label="AOT")
plt.scatter(x=lons_miss,y=lats_miss,c='gray',s=4,alpha=1,transform=ccrs.PlateCarree())
plt.text(0,0,"Gray points are missing values", fontsize=20,bbox=dict(facecolor='red', alpha=1))
plt.savefig("raw_data_viz_regridded.png")
plt.show()